In [2]:
import re
import varname
import warnings
import numpy as np
import MDAnalysis as mda
from matplotlib import pyplot as plt
from MDAnalysis.lib.distances import calc_angles
from MDAnalysis.lib.distances import calc_dihedrals

In [71]:
class AmbeRouxWoo(mda.core.universe.Universe):
    
    def __init__(self, topfile, receptor, ligand):
        
        super(AmbeRouxWoo, self).__init__(topfile)
        self.guess_missing_attr()
        self.topfile = topfile
        self.receptor = receptor
        self.ligand = ligand
        self.rec = self.select_atoms(self.receptor)
        self.lig = self.select_atoms(self.ligand)
        self.krec = 10.0
        self.klig = 10.0
        self.kTheta = 100.0
        self.kPhi = 100.0
        self.kPsi = 100.0
        self.ktheta = 100.0
        self.kphi = 100.0
        self.kdist = 2.0
        
    
    
    def guess_missing_attr(self):
        if hasattr(self.atoms,'elements'):
            return()
        else: 
            guessed_elements = guess_types(self.atoms.names)
            self.add_TopologyAttr('elements', guessed_elements)
            warnings.warn('Topology does not contain element information, so they will be \
                           guessed by MDAnalysis. Proceed with caution.')
            
    def calc_rgyr(self, attr):
        ag = getattr(self, attr)
        # get squared distance from center
        ri = ag.positions - ag.center_of_mass()
        # total mass calculation for the predefined atomgroup
        totmass = np.sum(ag.masses)
        # weight positions
        r_gyr = np.sqrt(np.sum(ag.masses*np.linalg.norm(ri, axis=1)**2)/totmass)
        # square root and return
        return r_gyr
        
    def find_anchor_groups(self, attr):
        
        ag = getattr(self,attr)
        com = ag.center_of_mass()
        # Calculate the radius of gyration; heuristic to find suitable atom anchors
        r_gyr = self.calc_rgyr(attr)
        f_rgyr = np.floor(r_gyr)
        c_rgyr = np.ceil(r_gyr)

        # define the COM position as a string for inputting it into the atom selection with a rgyr-dist criterion
        com_str = " ".join(ag.center_of_mass().astype(str))
        ag_rgyr = ag.select_atoms('point {} {} and not point {} {}'.format(com_str, c_rgyr, com_str, f_rgyr))
        
        # get indices of most orthogonal atoms of the ag_rgyr
        orthoi, orthoj, ortho = 0,0,100
        for i,ag1 in enumerate(ag_rgyr):
            for j,ag2 in enumerate(ag_rgyr): 
                angle = calc_angles(ag1.residue.atoms.center_of_mass(),com,ag2.residue.atoms.center_of_mass())
                if abs(angle - 0.5*np.pi) < ortho:
                    ortho = abs(angle - 0.5*np.pi)
                    orthoi,orthoj = i,j
        # save the residues of said atoms in an atom group and return them
        ag_com1 = ag_rgyr[orthoi].residue.atoms
        ag_com2 = ag_rgyr[orthoj].residue.atoms
        return(ag_com1,ag_com2)
    
    def set_anchors(self):
        for attr in ['rec','lig']:
            try:
                ag0 = getattr(self, attr)
                ag1, ag2 = self.find_anchor_groups(attr)
                ag_com0 = ag0.center_of_mass()
                ag_com1 = ag1.center_of_mass()
                ag_com2 = ag2.center_of_mass()
                ag_anchor = {'0': ag0,
                             '1': ag1,
                             '2': ag2,
                             'com0': ag_com0,
                             'com1': ag_com1,
                             'com2': ag_com2,
                            }
                setattr(self, attr+'_anchor', ag_anchor)
            except ValueError:
                warnings.warn('self.{} is not defined and no anchors could be set. Ignore \
                               this message if this is intentional for the bulk simulations.'.format(attr)) 
        
        
    def save_anchor_view(self, anchorsPDB='anchors.pdb', ligrecPDB='lig_rec.pdb'):
        u_anchor = mda.Universe.empty(n_atoms = 6,
                         n_residues = 6,
                         atom_resindex = [0,1,2,3,4,5],
                         residue_segindex = [0]*6,
                         trajectory = True)
        
        u_anchor.add_TopologyAttr('name', ['lig0', 'lig1', 'lig2','rec0', 'rec1', 'rec2'])
        u_anchor.add_TopologyAttr('type', ['Dummy', 'Dummy', 'Dummy','Dummy', 'Dummy', 'Dummy'])
        u_anchor.add_TopologyAttr('resname', ['lig0', 'lig1', 'lig2','rec0', 'rec1', 'rec2'])
        u_anchor.add_TopologyAttr('segid', ['DUM'])
        u_anchor.add_TopologyAttr('chainID', ['A']*6)
        
        u_anchor.select_atoms('name lig0').positions = self.lig_anchor['com0']
        u_anchor.select_atoms('name lig1').positions = self.lig_anchor['com1']
        u_anchor.select_atoms('name lig2').positions = self.lig_anchor['com2']
        u_anchor.select_atoms('name rec0').positions = self.rec_anchor['com0']
        u_anchor.select_atoms('name rec1').positions = self.rec_anchor['com1']
        u_anchor.select_atoms('name rec2').positions = self.rec_anchor['com2']
        u_anchor.atoms.write(anchorsPDB)
        
        self.select_atoms('not (resname WAT or resname Na+ or resname Cl-)').write(ligrecPDB)
        
        
    def parameterise_MULTI_RMSD(self, attr):
        anchor = getattr(self,attr+'_anchor')
        cv_i =','.join(anchor['0'].ids.astype(str))
        cv_r =','.join(anchor['0'].positions.reshape(-1).astype(str))
        cv_ni = anchor['0'].n_atoms
        cv_nr = anchor['0'].n_atoms*3
        return cv_i, cv_r, cv_ni, cv_nr
    
    
    def parameterise_COM_ANGLE(self,ag0,ag1,ag2):
        ''' Is supposed to generate amber cv_in strings for the following angle:
              1---2    
             /angle
            0
        '''
        com0 = ag0.center_of_mass()
        com1 = ag1.center_of_mass()
        com2 = ag2.center_of_mass()

        # MDA function to calculate angle of three positions
        angle = np.round(calc_angles(com0,com1,com2),3)
        # 300 kcal/mol/radian**2 = 0.1 kcal/mol/degree**2 from chipot et roux; seems pretty high though
        # 3 because of the three 0's that are part of the cv_i statement
        angle_ni = 3 + ag0.n_atoms + ag1.n_atoms + ag2.n_atoms
        angle_i0 = ",".join(ag0.ids.astype(str))
        angle_i1 = ",".join(ag1.ids.astype(str))
        angle_i2 = ",".join(ag2.ids.astype(str))
        return angle,angle_ni,angle_i0,angle_i1,angle_i2

    
    def parameterise_COM_TORSION(self, ag0, ag1, ag2, ag3):
        ''' Is supposed to generate amber cv_in strings for the following dihedral:
                  2---3
                 /dihdrl
            0---1
        '''
        # define center of masses from the atom groups 
        # (they are already known to the class, but code wise this is just cleater)
        com0 = ag0.center_of_mass()
        com1 = ag1.center_of_mass()
        com2 = ag2.center_of_mass()
        com3 = ag3.center_of_mass()

        # MDA function to calculate the dihedral between 4 coordinates
        tors = np.round(calc_dihedrals(com0, com1, com2, com3),3)
        
        # 4 because of the three 0's that are part of the cv_i statement
        tors_ni = (4 + ag0.n_atoms 
                     + ag1.n_atoms
                     + ag2.n_atoms 
                     + ag3.n_atoms)
        tors_i0 = ",".join(ag0.ids.astype(str))
        tors_i1 = ",".join(ag1.ids.astype(str))
        tors_i2 = ",".join(ag2.ids.astype(str))
        tors_i3 = ",".join(ag3.ids.astype(str))
        # output the torsion parameters for further use (e.g. by write_COM_TORSION method)
        return tors, tors_ni, tors_i0, tors_i1, tors_i2, tors_i3
    
    def write_MULTI_RMSD(self, attr, file, k_rmsd=0):
        cv_i, cv_r, cv_ni, cv_nr = self.parameterise_MULTI_RMSD(attr)
        file.write('&colvar ! {} RMSD Restraints\n'.format(attr.upper()))
        file.write("\tcv_type = 'MULTI_RMSD'\n")
        file.write('\tcv_ni = {}, cv_nr = {},\n'.format(cv_ni, cv_nr))
        file.write('\tcv_i = {},\n'.format(cv_i))
        file.write('\tcv_r = {},\n'.format(cv_r))
        file.write('\tanchor_position = -4.0, 1.0, 1.0, 5.0,\n')
        file.write('\tanchor_strength = {}, {},/\n'.format(k_rmsd,k_rmsd))
        
    def write_COM_ANGLE(self, file, ang_name, k_ang=0):
        
        if ang_name == 'Theta': 
            ang_type = 'Theta Orientational Restraints'
            ang, ang_ni, ang_i0, ang_i1, ang_i2 = self.parameterise_COM_ANGLE(self.lig_anchor['1'],
                                                                              self.lig_anchor['0'],
                                                                              self.rec_anchor['0'])
        elif ang_name == 'theta': 
            ang_type = 'theta Spherical Restraints'
            ang, ang_ni, ang_i0, ang_i1, ang_i2 = self.parameterise_COM_ANGLE(self.rec_anchor['1'],
                                                                              self.rec_anchor['0'],
                                                                              self.lig_anchor['0'])
        else:
            raise ValueError('{} isnt a valid option for ang_name. Either use "Theta" for ori., angular restraints or "theta" for sph., angular restraints.'.format(ang_name))
            
        low = np.round(ang-1.5,3)
        up = np.round(ang+1.5,3)

        file.write('&colvar ! {}\n'.format(ang_type))
        file.write("\tcv_type = 'COM_ANGLE'\n")
        file.write('\tcv_ni = {},\n'.format(ang_ni))
        file.write('\tcv_i = {},0,{},0,{},0,\n'.format(ang_i0, ang_i1, ang_i2))
        file.write('\tanchor_position = {}, {}, {}, {},\n'.format(low, ang, ang, up))
        file.write('\tanchor_strength = {}, {},/\n'.format(k_ang,k_ang))
        
    def write_COM_TORSION(self, file, tors_name, k_tors=0):
        # if else clauses to distinguish between the different torsions necessary for woo roux method
        if tors_name == 'Phi': 
            tors_type = 'Phi Orientational Restraints'
            tors, tors_ni, tors_i0, tors_i1, tors_i2, tors_i3 = self.parameterise_COM_TORSION(self.rec_anchor['0'],
                                                                                              self.lig_anchor['0'],
                                                                                              self.lig_anchor['1'],
                                                                                              self.lig_anchor['2'])
        elif tors_name == 'Psi': 
            tors_type = 'Psi Orientational Restraints'
            tors, tors_ni, tors_i0, tors_i1, tors_i2, tors_i3 = self.parameterise_COM_TORSION(self.rec_anchor['1'],
                                                                                              self.rec_anchor['0'],
                                                                                              self.lig_anchor['0'],
                                                                                              self.lig_anchor['1'])
        elif tors_name == 'phi': 
            tors_type = 'phi Spherical Restraints'
            tors, tors_ni, tors_i0, tors_i1, tors_i2, tors_i3 = self.parameterise_COM_TORSION(self.rec_anchor['2'],
                                                                                              self.rec_anchor['1'],
                                                                                              self.rec_anchor['0'],
                                                                                              self.lig_anchor['0'])
        else:
            raise ValueError('{} isnt a valid option for torsname. Either use "Phi"/"Psi" for ori., angular restraints or "phi" for sph., angular restraints.'.format(tors_name))
            
        low = np.round(tors-1.5,3)
        up = np.round(tors+1.5,3)

        file.write('&colvar ! {}\n'.format(tors_type))
        file.write("\tcv_type = 'COM_TORSION'\n")
        file.write('\tcv_ni = {},\n'.format(tors_ni))
        file.write('\tcv_i = {},0,{},0,{},0,{},0,\n'.format(tors_i0, tors_i1, tors_i2, tors_i3))
        file.write('\tanchor_position = {}, {}, {}, {},\n'.format(low, tors, tors, up))
        file.write('\tanchor_strength = {}, {},/\n'.format(k_tors,k_tors))
    
    
    def write_COM_DIST(self, file, k_dist=0):
        
        com_ni = self.lig_anchor['0'].n_atoms + self.rec_anchor['0'].n_atoms
        com_i0 = ','.join(self.lig_anchor['0'].ids.astype(str))
        com_i1 = ','.join(self.rec_anchor['0'].ids.astype(str))
        
        file.write('&colvar ! R_COM between Ligand and Receptor\n')
        file.write("\tcv_type = 'COM_DISTANCE'\n")
        file.write('\tcv_ni = {},\n'.format(com_ni))
        file.write('\tcv_i = {},0,{},0\n'.format(com_i0, com_i1))
        file.write('\tanchor_position = {}, {}, {}, {},\n'.format(0, 'WINDOW', 'WINDOW', 99))
        file.write('\tanchor_strength = {}, {},/\n'.format(k_dist, k_dist))
    
    def write_restraint_file(self,filename, lig=None, rec=None, ori=None, ang=None, com=None):
        
        with open(filename, 'w') as file:
            match(lig):
                case True : self.write_MULTI_RMSD('lig',file,self.klig)
                case False: self.write_MULTI_RMSD('lig',file)
                case None : pass

            match(rec):
                case True : self.write_MULTI_RMSD('rec',file, self.krec)
                case False: self.write_MULTI_RMSD('rec',file)
                case None : pass

            match(ori):
                case True:
                    self.write_COM_ANGLE(file, 'Theta', k_ang=self.kTheta)
                    self.write_COM_TORSION(file, 'Phi', k_tors=self.kPhi)
                    self.write_COM_TORSION(file, 'Psi', k_tors=self.kPsi)
                case False:
                    self.write_COM_ANGLE(file, 'Theta')
                    self.write_COM_TORSION(file, 'Phi')
                    self.write_COM_TORSION(file, 'Psi')
                case None:
                    pass

            match(ang):
                case True:
                    self.write_COM_ANGLE(file, 'theta', k_ang=self.ktheta)
                    self.write_COM_TORSION(file, 'phi', k_tors=self.kphi)
                case False:
                    self.write_COM_ANGLE(file, 'theta')
                    self.write_COM_TORSION(file, 'phi')
                case None:
                    pass

            match(com):
                case  True: self.write_COM_DIST(file,k_dist=self.kdist)
                case False: self.write_COM_DIST(file)
                case  None: pass
            # both these strings are unnecessary, but the simulations worked and I am too afraid to remove them. later maybe ...
            file.write('end\n')
            file.write('eof\n')

In [74]:
bulkpeptide = AmbeRouxWoo('../_sim/05_TIP3P_med_peptideBulk/_output/tip3p_medconc_10A_leap.pdb', receptor=None, ligand='protein and not element H')
bulkpeptide.set_anchors()
bulkpeptide.write_restraint_file("../_inp/pB.r", True)

<AtomGroup [<Atom 1: N of type N of resname ASN, resid 1 and segid SYSTEM and altLoc >, <Atom 5: CA of type C of resname ASN, resid 1 and segid SYSTEM and altLoc >, <Atom 7: CB of type C of resname ASN, resid 1 and segid SYSTEM and altLoc >, ..., <Atom 348: C of type C of resname THR, resid 19 and segid SYSTEM and altLoc >, <Atom 349: O of type O of resname THR, resid 19 and segid SYSTEM and altLoc >, <Atom 350: OXT of type O of resname THR, resid 19 and segid SYSTEM and altLoc >]>


/tmp/ipykernel_2425553/539429609.py:29: UserWarning: Topology does not contain element information, so they will be                            guessed by MDAnalysis. Proceed with caution.
  warnings.warn('Topology does not contain element information, so they will be \
/tmp/ipykernel_2425553/539429609.py:86: UserWarning: self.rec is not defined and no anchors could be set. Ignore                                this message if this is intentional for the bulk simulations.
  warnings.warn('self.{} is not defined and no anchors could be set. Ignore \


In [11]:
test1 = AmbeRouxWoo('../_inp/1a4t_xray.pdb', '(nucleic or resname G5 or resname C3) and not element H', 'protein and not element H')

In [12]:
test1.set_anchors()
test1.save_anchor_view()
test1.write_restraint_file("./_.r",False, False, False, False, False)
test1.write_restraint_file("./_p.r",True, False, False, False, False)
test1.write_restraint_file("./_pn.r",True, True, False, False, False)
test1.write_restraint_file("./_pno.r",True, True, True, False, False)
test1.write_restraint_file("./_pnoa.r",True, True, True, True, False)
test1.write_restraint_file("./_pnoaus.r",True, True, True, True,True)

/home/luisvollmers/anaconda3/envs/mdanal/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:753: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/luisvollmers/anaconda3/envs/mdanal/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/luisvollmers/anaconda3/envs/mdanal/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  warnings.warn("Found no information for attr: '{}'"
/home/luisvollmers/anaconda3/envs/mdanal/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'resids' Using default value of '1'
  warnings.warn("Found no information for attr: '{}'"
/home/luisvollmers/anaconda3/envs/mdanal/

In [44]:
bulkpeptide = AmbeRouxWoo('../_inp/1a4t_peptide_ready.pdb', receptor=None, ligand='protein and not element H')
bulkpeptide.set_anchors()
bulkpeptide.write_restraint_file("../_inp/pB.r", True)

/tmp/ipykernel_1074884/1498513328.py:77: UserWarning: receptor/ligand is not defined and no anchors could be set. Ignore this message if this is intentional for the bulk simulations.
  warnings.warn('receptor/ligand is not defined and no anchors could be set. Ignore this message if this is intentional for the bulk simulations.')


In [8]:
bulkpeptide = AmbeRouxWoo('../_sim/05_TIP3P_med_peptideBulk/_output/tip3p_medconc_10A_leap.pdb', receptor=None, ligand='protein and not element H')
bulkpeptide.set_anchors()
bulkpeptide.write_restraint_file("../_inp/pB_bb.r", True)

/home/luisvollmers/anaconda3/envs/mdanal/lib/python3.10/site-packages/MDAnalysis/core/groups.py:3207: UserWarning: Empty string to select atoms, empty group returned.
  warnings.warn("Empty string to select atoms, empty group returned.",
/tmp/ipykernel_1074884/1498513328.py:77: UserWarning: receptor/ligand is not defined and no anchors could be set. Ignore this message if this is intentional for the bulk simulations.
  warnings.warn('receptor/ligand is not defined and no anchors could be set. Ignore this message if this is intentional for the bulk simulations.')


In [75]:
bulknucleic = AmbeRouxWoo('../_sim/06_TIP3P_med_rnaBulk/_output/tip3p_medconc_10A_leap.pdb', receptor='(nucleic or resname G5 or resname C3) and not element H', ligand=None)
bulknucleic.set_anchors()
bulknucleic.write_restraint_file("../_inp/nB.r", None, True)

/home/luisvollmers/anaconda3/envs/mdanal/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:328: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/tmp/ipykernel_2425553/539429609.py:29: UserWarning: Topology does not contain element information, so they will be                            guessed by MDAnalysis. Proceed with caution.
  warnings.warn('Topology does not contain element information, so they will be \
/home/luisvollmers/anaconda3/envs/mdanal/lib/python3.10/site-packages/MDAnalysis/core/groups.py:3207: UserWarning: Empty string to select atoms, empty group returned.
  warnings.warn("Empty string to select atoms, empty group returned.",
/tmp/ipykernel_2425553/539429609.py:86: UserWarning: self.lig is not defined and no anchors could be set. Ignore                                this message if 